In [1]:
# ==========================================
# 今日头条热榜 -> 千问分析 -> 利多/利空展示
# 稳定版：JSON-only + 本地严格校验 + 自动重试（最多N次）
# 新增：
#  1) 更稳两处小改动（提高重试次数 + 禁止字符串里出现 ASCII 双引号）
#  2) 可选“弹出窗口”（用浏览器打开本地 HTML 展示两个 DF）
#  3) 每次运行保存 Excel（同一文件两张 sheet：利多/利空；文件名=时间戳，自动避免重名）
# ==========================================

# =========================
# A) 参数区（你主要改这里）
# =========================
CONFIG = {
    # 定时运行
    "ENABLE_SCHEDULER": False,    # True=循环定时跑；False=只跑一次
    "INTERVAL_MINUTES": 30,       # 建议 10~20 也可以

    # 热搜抓取
    "TOP_K": 30,
    "TOUTIAO_TIMEOUT": 12,

    # 千问（OpenAI 兼容接口）
    "DASHSCOPE_API_KEY": "sk-08ab985707514fd88c651a59b6de33c7",
    "QWEN_MODEL": "qwen-plus",
    "QWEN_BASE_URL": "https://dashscope.aliyuncs.com/compatible-mode/v1",
    "QWEN_TIMEOUT": 60,

    # 稳定输出建议温度 0
    "TEMPERATURE": 0.0,
    "MAX_TOKENS": 1200,

    # 输出控制
    "MAX_ITEMS_EACH_SIDE": 12,

    # ✅ 小改动 1：提高重试次数（更稳）
    "MAX_QWEN_RETRIES": 10,
    "RETRY_SLEEP_SECONDS": 0.6,

    # 调试
    "DEBUG_SHOW_RAW_ON_FAIL": True,   # 最终失败时打印原始输出片段

    # 结果展示：弹出窗口（浏览器新窗口/新标签打开 HTML）
    "ENABLE_POPUP": True,            # True 时每次运行弹出 HTML 窗口展示两个 DF
    "POPUP_TITLE_PREFIX": "热搜利多/利空分析",

    # 结果保存：Excel
    "ENABLE_SAVE_EXCEL": True,
    "OUTPUT_DIR": "./hotsearch_stock_outputs",   # 保存目录
}

# =========================
# B) 依赖
# =========================
import os
import time
import json
import re
import webbrowser
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional

import requests
import pandas as pd

try:
    from IPython.display import display  # type: ignore
except Exception:
    display = None

# DataFrame 显示：全部行列 + 完整字符串（你要求显示全部内容）
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)


# =========================
# 1) 抓取今日头条热榜
# =========================
def fetch_toutiao_hot(top_k: int = 30, timeout: int = 12) -> List[Dict[str, Any]]:
    url = "https://www.toutiao.com/hot-event/hot-board/?origin=toutiao_pc"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
        ),
        "Referer": "https://www.toutiao.com/",
        "Accept": "application/json,text/plain,*/*",
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    payload = r.json()

    data = payload.get("data")
    if not isinstance(data, list):
        raise ValueError(f"Unexpected response schema. Top keys: {list(payload.keys())}")

    items: List[Dict[str, Any]] = []
    for i, it in enumerate(data[:top_k], start=1):
        title = it.get("Title") or it.get("title") or ""
        hot_value = it.get("HotValue") or it.get("hot_value") or 0
        link = it.get("Url") or it.get("url") or ""

        try:
            hot_value_num = float(hot_value)
        except Exception:
            hot_value_num = None

        items.append({
            "rank": i,
            "title": str(title).strip(),
            "hot_value": hot_value_num,
            "url": str(link).strip(),
        })

    return [x for x in items if x["title"]]


# =========================
# 2) 千问调用（OpenAI 兼容 chat/completions）
# =========================
def qwen_chat_raw(
    api_key: str,
    base_url: str,
    model: str,
    messages: List[Dict[str, str]],
    timeout: int,
    temperature: float,
    max_tokens: int,
) -> Tuple[str, Dict[str, Any]]:
    """
    返回 (text, raw_json)
    """
    endpoint = base_url.rstrip("/") + "/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

    body: Dict[str, Any] = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
        # 尽量提示输出 JSON（核心仍靠重试 + prompt 约束 + 本地校验）
        "response_format": {"type": "json_object"},
    }

    r = requests.post(endpoint, headers=headers, json=body, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:1500]}")
    raw = r.json()

    msg = raw["choices"][0]["message"]

    # 有些实现会走 tool_calls arguments
    tool_calls = msg.get("tool_calls")
    if isinstance(tool_calls, list) and tool_calls:
        fn = tool_calls[0].get("function", {})
        args = fn.get("arguments")
        if isinstance(args, str) and args.strip():
            return args, raw

    content = msg.get("content")
    if isinstance(content, str):
        return content, raw

    return json.dumps(msg, ensure_ascii=False), raw


# =========================
# 3) JSON 清理/解析/校验
# =========================
def clean_text(s: str) -> str:
    s = s.strip()

    # 去掉 ```json``` 包裹
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)

    # 中文引号 -> 英文引号（避免解析失败）
    s = s.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    # JSON 不允许 NaN/Infinity
    s = re.sub(r"\bNaN\b", "null", s)
    s = re.sub(r"\bInfinity\b", "null", s)
    s = re.sub(r"\b-Infinity\b", "null", s)

    # 尾逗号（常见）
    s = re.sub(r",\s*([}\]])", r"\1", s)

    return s


def strict_parse_json(s: str) -> Dict[str, Any]:
    """
    严格：必须是 JSON object；并且清理后必须以 { 开头、} 结尾（防止夹带文本）
    """
    s2 = clean_text(s)
    s2_strip = s2.strip()
    if not (s2_strip.startswith("{") and s2_strip.endswith("}")):
        raise ValueError("Output is not a pure JSON object (missing { } boundaries).")

    obj = json.loads(s2_strip)
    if not isinstance(obj, dict):
        raise ValueError("JSON is not an object.")
    return obj


def validate_result(obj: Dict[str, Any]) -> None:
    for k in ["bullish", "bearish", "ignored"]:
        if k not in obj:
            raise ValueError(f"Missing key: {k}")
        if not isinstance(obj[k], list):
            raise ValueError(f"Key '{k}' must be a list.")


# =========================
# 4) Prompt（更硬约束 JSON-only）
#    ✅ 小改动 2：禁止字符串里出现 ASCII 双引号 "
# =========================
def build_messages(items: List[Dict[str, Any]], max_each_side: int) -> List[Dict[str, str]]:
    hot_lines = []
    for it in items:
        hv = it["hot_value"]
        hv_str = f"{hv:.0f}" if isinstance(hv, (int, float)) and hv is not None else ""
        hot_lines.append(f'{it["rank"]}. {it["title"]} (HotValue={hv_str})')

    user = f"""
你会收到“今日头条热榜标题列表”。请判断哪些热搜可能引起行业/板块层面的短期股价波动。
将其分为：bullish（利多）、bearish（利空）、ignored（不太相关）。

极其重要的输出要求（必须遵守）：
- 只输出一段严格 JSON（不要 markdown、不要解释、不要任何前后缀文字）。
- 输出必须以字符 "{{" 开头，并以 "}}" 结尾。
- JSON 中不得出现 NaN/Infinity。
- confidence 必须是 0~1 的小数（不要百分号）。
- 每组最多 {max_each_side} 条，优先挑选“政策/监管、宏观数据、事故风险、重大交易、科技突破、能源/大宗、医疗/药品、金融风险”等更可能影响股价的主题。
- 不要编造具体股票代码；只给行业/板块。
- 【重要】所有字符串字段（topic/mechanism/reason/industries）中禁止出现 ASCII 双引号字符 (")。
  如果需要表达引用，请用中文引号（“ ”）或括号，或用单引号（'）代替。

固定 JSON Schema（字段名必须一致）：
{{
  "bullish": [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "bearish":  [{{"rank": 1, "topic": "...", "industries": ["..."], "mechanism": "...", "confidence": 0.0}}],
  "ignored":  [{{"rank": 1, "topic": "...", "reason": "..."}}]
}}

热榜标题列表：
{chr(10).join(hot_lines)}
""".strip()

    return [
        {"role": "system", "content": "你是严谨的中文金融信息分析助手。必须输出严格 JSON。"},
        {"role": "user", "content": user},
    ]


# =========================
# 5) 自动重试：直到拿到可解析 JSON 或超出最大次数
# =========================
def qwen_analyze_with_retries(items: List[Dict[str, Any]]) -> Dict[str, Any]:
    messages = build_messages(items, CONFIG["MAX_ITEMS_EACH_SIDE"])

    last_text: Optional[str] = None
    last_err: Optional[Exception] = None

    for attempt in range(1, CONFIG["MAX_QWEN_RETRIES"] + 1):
        try:
            text, raw = qwen_chat_raw(
                api_key=CONFIG["DASHSCOPE_API_KEY"],
                base_url=CONFIG["QWEN_BASE_URL"],
                model=CONFIG["QWEN_MODEL"],
                messages=messages,
                timeout=CONFIG["QWEN_TIMEOUT"],
                temperature=CONFIG["TEMPERATURE"],
                max_tokens=CONFIG["MAX_TOKENS"],
            )
            last_text = text

            obj = strict_parse_json(text)
            validate_result(obj)
            return obj

        except Exception as e:
            last_err = e
            time.sleep(CONFIG["RETRY_SLEEP_SECONDS"])

    # 全失败：输出最后一次原始文本片段辅助调试
    if CONFIG.get("DEBUG_SHOW_RAW_ON_FAIL") and last_text:
        print("\n[DEBUG] Last model output (first 1200 chars):")
        print(clean_text(last_text)[:1200])

    raise RuntimeError(
        f"Failed to obtain valid JSON after {CONFIG['MAX_QWEN_RETRIES']} attempts. Last error: {last_err}"
    )


# =========================
# 6) DataFrame + 展示
# =========================
def to_df(lst: List[Dict[str, Any]]) -> pd.DataFrame:
    if not lst:
        return pd.DataFrame()
    df = pd.DataFrame(lst).copy()
    if "industries" in df.columns:
        df["industries"] = df["industries"].apply(lambda x: "，".join(x) if isinstance(x, list) else x)
    return df


def show_df(df: pd.DataFrame):
    if display is not None:
        display(df)
    else:
        print(df.to_string(index=False))


# =========================
# 7) 弹出窗口展示（浏览器打开本地 HTML）
# =========================
def popup_show_dfs(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, title: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    def _df_to_html(df: pd.DataFrame) -> str:
        if df is None or df.empty:
            return "<p><em>（无数据）</em></p>"
        return df.to_html(index=False, escape=True)

    html = f"""
<!doctype html>
<html lang="zh">
<head>
<meta charset="utf-8">
<title>{title}</title>
<style>
body {{ font-family: -apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Arial; margin: 20px; }}
h1 {{ margin-bottom: 12px; }}
h2 {{ margin-top: 22px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f6f6f6; }}
tr:nth-child(even) {{ background: #fafafa; }}
.small {{ color: #666; font-size: 12px; }}
</style>
</head>
<body>
<h1>{title}</h1>
<div class="small">生成时间：{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</div>

<h2>📈 可能利多（正面催化）</h2>
{_df_to_html(bullish_df)}

<h2>📉 可能利空（负面冲击）</h2>
{_df_to_html(bearish_df)}

</body>
</html>
""".strip()

    # 用时间戳命名，避免覆盖
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.abspath(os.path.join(output_dir, f"popup_{ts}.html"))
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)

    # 打开浏览器（通常会新标签/新窗口；浏览器设置可能影响行为）
    webbrowser.open(f"file://{filepath}", new=1)


# =========================
# 8) 保存 Excel（同一文件两张 sheet：利多/利空；文件名=时间戳，自动避免重名）
# =========================
def save_dfs_to_excel(bullish_df: pd.DataFrame, bearish_df: pd.DataFrame, ts_str: str, output_dir: str) -> str:
    os.makedirs(output_dir, exist_ok=True)

    base_name = f"hotsearch_{ts_str}.xlsx"
    path = os.path.join(output_dir, base_name)

    # 避免重名：如果存在则追加 _1, _2 ...
    if os.path.exists(path):
        i = 1
        while True:
            alt = os.path.join(output_dir, f"hotsearch_{ts_str}_{i}.xlsx")
            if not os.path.exists(alt):
                path = alt
                break
            i += 1

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        # sheet 名按你的要求：利多/利空
        (bullish_df if bullish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利多", index=False)
        (bearish_df if bearish_df is not None else pd.DataFrame()).to_excel(writer, sheet_name="利空", index=False)

    return os.path.abspath(path)


# =========================
# 9) 运行一次
# =========================
def run_once() -> Dict[str, Any]:
    items = fetch_toutiao_hot(CONFIG["TOP_K"], CONFIG["TOUTIAO_TIMEOUT"])
    result = qwen_analyze_with_retries(items)

    ts_display = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ts_fname = datetime.now().strftime("%Y%m%d_%H%M%S")

    bullish_df = to_df(result.get("bullish", []))
    bearish_df = to_df(result.get("bearish", []))

    print(f"\n===== Run @ {ts_display} =====")
    print(f"Fetched {len(items)} hot topics. "
          f"Model returned {len(result.get('bullish', []))} bullish / {len(result.get('bearish', []))} bearish.\n")

    print("📈 可能利多（正面催化）")
    show_df(bullish_df)

    print("\n📉 可能利空（负面冲击）")
    show_df(bearish_df)

    # 保存 Excel
    if CONFIG.get("ENABLE_SAVE_EXCEL"):
        saved_path = save_dfs_to_excel(
            bullish_df=bullish_df,
            bearish_df=bearish_df,
            ts_str=ts_fname,
            output_dir=CONFIG["OUTPUT_DIR"],
        )
        print(f"\n💾 已保存 Excel：{saved_path}")

    # 弹出窗口（浏览器）
    if CONFIG.get("ENABLE_POPUP"):
        title = f"{CONFIG.get('POPUP_TITLE_PREFIX', '热搜分析')} - {ts_display}"
        popup_show_dfs(bullish_df, bearish_df, title=title, output_dir=CONFIG["OUTPUT_DIR"])

    return result


# =========================
# 10) 定时循环（可选）
# =========================
def run_scheduler():
    interval_sec = int(CONFIG["INTERVAL_MINUTES"] * 60)
    print(f"Scheduler started. Interval={CONFIG['INTERVAL_MINUTES']} minutes. Stop with Kernel Interrupt (⛔).")
    while True:
        try:
            run_once()
        except Exception as e:
            print(f"[ERROR] {e}")
        time.sleep(interval_sec)


# =========================
# 11) 执行入口
# =========================
_ = run_once()

if CONFIG["ENABLE_SCHEDULER"]:
    run_scheduler()



===== Run @ 2025-12-28 10:53:28 =====
Fetched 30 hot topics. Model returned 1 bullish / 5 bearish.

📈 可能利多（正面催化）


,rank,topic,industries,mechanism,confidence
0,3,2025中国经济盘点,宏观经济，资本市场，消费，制造业,对2025年经济形势的系统性回顾与展望可能增强市场信心，若传递积极信号将利好整体风险资产定价,0.75



📉 可能利空（负面冲击）


,rank,topic,industries,mechanism,confidence
0,9,中方为何反制20家美军火商,国防军工，外贸出口，高端制造,对美军事企业实施反制可能引发贸易摩擦升级担忧，影响相关产业链国际合作与订单稳定性,0.80
1,7,6.6级地震致台北摇晃近1分钟,半导体，电子代工，基础设施,台湾地区为全球半导体重要生产基地，强震可能引发产能中断担忧，冲击供应链预期,0.70
2,21,媒体：南博前院长徐湖平被带走调查,文化艺术，文物交易，拍卖行业,高级文化官员被查或引发艺术品市场合规审查预期上升，抑制非标资产交易情绪,0.65
3,24,老人存167万消失突发脑梗后离世,银行业，财富管理，金融消费者保护,大额存款失踪事件若暴露银行内控漏洞，可能引发储户对资金安全的普遍担忧,0.60
4,25,新治安管理处罚法对你我生活有何影响,互联网平台，在线内容，安防科技,若新法强化网络行为监管，可能增加平台运营合规成本，限制用户生成内容扩张,0.55



💾 已保存 Excel：/Users/pz/VScode/Stock_Ana/hotsearch_stock_outputs/hotsearch_20251228_105328.xlsx
